# Pytorch tutorial for image classification
Checkout the Pytorch tutorial in this [link](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

## Loading the Data (CIFAR10)
Downloading the data, normalize it and transform to tensors, and wrap it in DataLoaders for 'train' and 'test'.

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## The Model
Define a neural network model.
The model is defined as a class which inherits from the pytorch class `Model`.
Notice that first, we define the layers in the `__init__` and only after we use the defined layers in the `forward`.
This writing is similar for tensorflow as well.

In [3]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

We define the optimizer and the loss function, which defines how to train our model:

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training
The training process:

In [7]:
from ToGrowABackbone.training_utils import evaluate

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
    evaluate(net, testloader, classes)

print('Finished Training')

[1,  2000] loss: 1.241
[1,  4000] loss: 1.235
[1,  6000] loss: 1.208
[1,  8000] loss: 1.198
[1, 10000] loss: 1.200
[1, 12000] loss: 1.186
Accuracy for class plane is: 54.1 %
Accuracy for class car   is: 64.8 %
Accuracy for class bird  is: 39.2 %
Accuracy for class cat   is: 45.8 %
Accuracy for class deer  is: 53.1 %
Accuracy for class dog   is: 36.2 %
Accuracy for class frog  is: 65.0 %
Accuracy for class horse is: 78.6 %
Accuracy for class ship  is: 71.2 %
Accuracy for class truck is: 69.4 %
[2,  2000] loss: 1.098
[2,  4000] loss: 1.118
[2,  6000] loss: 1.130
[2,  8000] loss: 1.117
[2, 10000] loss: 1.136
[2, 12000] loss: 1.122
Accuracy for class plane is: 69.2 %
Accuracy for class car   is: 71.6 %
Accuracy for class bird  is: 45.1 %
Accuracy for class cat   is: 42.4 %
Accuracy for class deer  is: 63.7 %
Accuracy for class dog   is: 46.7 %
Accuracy for class frog  is: 65.3 %
Accuracy for class horse is: 65.6 %
Accuracy for class ship  is: 53.9 %
Accuracy for class truck is: 69.4 %
Fini

# Evaluation
Evaluate our model on the test set.

In [6]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

Accuracy for class plane is: 64.2 %
Accuracy for class car   is: 60.1 %
Accuracy for class bird  is: 61.5 %
Accuracy for class cat   is: 41.0 %
Accuracy for class deer  is: 21.2 %
Accuracy for class dog   is: 16.5 %
Accuracy for class frog  is: 71.3 %
Accuracy for class horse is: 62.9 %
Accuracy for class ship  is: 65.6 %
Accuracy for class truck is: 60.3 %
